# Data processing and training 

##### Installing libraries and importing packages

In [6]:
!pip install accelerate==0.17.0
!pip install transformers==4.28.0
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 6.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 1

In [7]:
# install sentencepiece
!pip install "sentencepiece==0.1.96"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.9 MB/s eta 0:00:00


In [9]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
import pandas as pd
import tensorflow as tf
import sentencepiece

In [11]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset
from sklearn.metrics import mean_squared_error

In [12]:
import evaluate

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/yt_analysis

/content/drive/MyDrive/Colab Notebooks/yt_analysis


In [ ]:
!pwd

/content/drive/MyDrive/Colab Notebooks/yt_analysis


In [ ]:
data = pd.read_csv('data_views.csv')

In [ ]:
data.head()

,title,views,tags,description,view_group
0,youtube channel 202004202304,96,0,NaN,"(79.0, 186.0]"
1,100 youtube channel 202304,66,mongolian youtube channel,100 youtube channel,"(-0.1, 79.0]"
2,mongolian youtube channel top5,318,mongolian youtube channel,youtube,"(306.0, 455.0]"
3,2x2,2248,NaN,2x2,"(2224.3, 3524.0]"
4,top 25,281,mongol tik tok mongolia tik tok,top list mongolia 3 tik tok mongolia,"(186.0, 306.0]"


In [ ]:
data = data.rename(columns={'title':'text', 'views':'labels'})

In [ ]:
data = data[['text', 'labels']]

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.head()

,text,labels
0,youtube channel 202004202304,96
1,100 youtube channel 202304,66
2,mongolian youtube channel top5,318
3,2x2,2248
4,top 25,281


In [ ]:
data['labels'] = data['labels'].astype(float)

# Splitting dataset

In [ ]:
dataset = Dataset.from_pandas(data, preserve_index=False)

In [ ]:
dataset = dataset.train_test_split(test_size=0.3)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 85772
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 36760
    })
})

## Loading pretrained model 'distilbert-base-uncase' and tokenizer

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122532 entries, 0 to 127225
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   text    122532 non-null  object 
 1   labels  122532 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.8+ MB


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/85772 [00:00<?, ? examples/s]

Map:   0%|          | 0/36760 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse":rmse}

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", 
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size = 16,
                                  per_device_eval_batch_size = 16,
                                  num_train_epochs=3, 
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end = False)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_datasets["train"],
    eval_dataset= tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rmse
1,9497593932266.128906,22405336530944.000000,4733427.500000
2,9492860735701.166016,22399516934144.000000,4732813.000000


Epoch,Training Loss,Validation Loss,Rmse
1,9497593932266.128906,22405336530944.000000,4733427.500000
2,9492860735701.166016,22399516934144.000000,4732813.000000
3,9489265973855.027344,22396754984960.000000,4732521.000000


TrainOutput(global_step=16083, training_loss=9493240213940.773, metrics={'train_runtime': 13122.5112, 'train_samples_per_second': 19.609, 'train_steps_per_second': 1.226, 'total_flos': 3.408537327748301e+16, 'train_loss': 9493240213940.773, 'epoch': 3.0})

### Save model

In [26]:
# save the model/tokenizer
model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")
# load the model/tokenizer
from transformers import AutoModelForTokenClassification
model = AutoModelForSequenceClassification.from_pretrained("model")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [15]:
from transformers import AutoModelForTokenClassification
model = AutoModelForSequenceClassification.from_pretrained("model")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [22]:
from transformers import Trainer
trainer = Trainer(model=pickled_model)
def tokenize_function(examples):
    return tokenizer_pkl(examples["text"], padding="max_length", truncation=True) 
def pipeline_prediction(text):
    df=pd.DataFrame({'text':[text]})
    dataset = Dataset.from_pandas(df,preserve_index=False) 
    tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return(raw_pred[0][0])

In [23]:
pipeline_prediction("How to make cheeseburger")


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

16844.059

In [ ]:
pickle.dump(model, open('model.pkl', 'wb'))
# saving
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer_pkl = pickle.load(handle)

In [ ]:
pickled_model = pickle.load(open('model.pkl', 'rb'))